# Data Cleaning

Simulating a "Dirty" Dataset for Cleaning Practice
Let’s start with the most common real-world issues:

- Missing values (NaNs)

- Duplicates

- Inconsistent casing / whitespace

- Wrong data types

- Outliers

- Invalid entries


🎯 Sample Problem Statement:

**"You’ve received user registration data from multiple sources. Clean the dataset before it is used for customer profiling and modeling."**

In [1]:
# Simulate the Dataset

import pandas as pd
import numpy as np

# Simulated dirty dataset
data = {
    'Customer_ID': [' C001 ', 'C002', 'C003', 'C004', 'C005', 'C005'],  # duplicate, whitespace
    'Name': ['John', 'Ana', 'mike', 'Sarah', np.nan, 'John'],         # NaN, duplicate, inconsistent casing
    'Age': [25, np.nan, -1, 45, 120, 30],                              # Missing, invalid age
    'Email': ['john@example.com', 'ana@gmail.com', 'mike@domain', 'sarah@company.com', '', 'john@example.com'],  # Invalid email, duplicate
    'Signup_Date': ['2023-01-01', '2023/01/02', '01-03-2023', '2023-01-04', 'Not Available', '2023-01-01'],       # Inconsistent dates
    'Income': ['40,000', '50000', 'fifty thousand', np.nan, '60000', '40000'],                                     # Strings, missing, inconsistent
}

df_dirty = pd.DataFrame(data)
df_dirty.head(5)

,Customer_ID,Name,Age,Email,Signup_Date,Income
0,C001,John,25.0,john@example.com,2023-01-01,"40,000"
1,C002,Ana,NaN,ana@gmail.com,2023/01/02,50000
2,C003,mike,-1.0,mike@domain,01-03-2023,fifty thousand
3,C004,Sarah,45.0,sarah@company.com,2023-01-04,NaN
4,C005,NaN,120.0,,Not Available,60000


Let’s Clean It Step by Step

➤ Missing values

➤ Duplicates

➤ String cleanup (case, whitespace)

➤ Invalid numeric values

➤ Date parsing

➤ Outlier detection

➤ Type correction

In [3]:
# ✅ Step 1: View the Raw Dataset
print(df_dirty)


  Customer_ID   Name    Age              Email    Signup_Date          Income
0       C001    John   25.0   john@example.com     2023-01-01          40,000
1        C002    Ana    NaN      ana@gmail.com     2023/01/02           50000
2        C003   mike   -1.0        mike@domain     01-03-2023  fifty thousand
3        C004  Sarah   45.0  sarah@company.com     2023-01-04             NaN
4        C005    NaN  120.0                     Not Available           60000
5        C005   John   30.0   john@example.com     2023-01-01           40000


In [4]:
# 🧪 Step 2: Remove Leading/Trailing Whitespaces

df_dirty['Customer_ID'] = df_dirty['Customer_ID'].str.strip()

# Why? In real datasets, extra spaces cause issues in matching or merging.

In [6]:
# 🔁 Step 3: Remove Duplicates

df_cleaned = df_dirty.drop_duplicates()

# 📌 Why? Duplicated rows distort statistics and model learning.

In [7]:
# 🧹 Step 4: Standardize Text Columns

df_cleaned['Name'] = df_cleaned['Name'].str.strip().str.title()

 # Why? Ensure consistency — "mike", "Mike", " M I K E " should all be the same

In [8]:
# Step 5: Handle Missing Values

# Check where missing
print(df_cleaned.isnull().sum())

# Fill missing name with "Unknown"
df_cleaned['Name'] = df_cleaned['Name'].fillna('Unknown')

# Fill missing income with median (after cleaning it)
# We'll handle Income separately later
# Why? ML models can’t handle null values without preprocessing.

Customer_ID    0
Name           1
Age            1
Email          0
Signup_Date    0
Income         1
dtype: int64


In [11]:
# 🚨 Step 6: Fix Invalid Values (Age)

# Replace invalid ages (< 0 or > 100) with median of valid range
valid_age = df_cleaned[(df_cleaned['Age'] > 0) & (df_cleaned['Age'] <= 100)]
median_age = valid_age['Age'].median()

df_cleaned['Age'] = df_cleaned['Age'].apply(lambda x: median_age if (x < 0 or x > 100 or pd.isnull(x)) else x)
#  Why? -1 and 120 are invalid for age, and missing values must be imputed.

In [10]:
df_cleaned.head()

,Customer_ID,Name,Age,Email,Signup_Date,Income
0,C001,John,25.0,john@example.com,2023-01-01,"40,000"
1,C002,Ana,30.0,ana@gmail.com,2023/01/02,50000
2,C003,Mike,30.0,mike@domain,01-03-2023,fifty thousand
3,C004,Sarah,45.0,sarah@company.com,2023-01-04,NaN
4,C005,Unknown,30.0,,Not Available,60000


In [13]:
# 📧 Step 7: Validate Emails (Simple Check)

df_cleaned['Email'] = df_cleaned['Email'].replace('', np.nan)

# Keep only emails that contain "@" and a "."
df_cleaned = df_cleaned[df_cleaned['Email'].str.contains('@') & df_cleaned['Email'].str.contains('.', regex=False)]

# Why? Invalid or missing emails are useless for communication or customer insights.

In [15]:
# 📅 Step 8: Clean & Convert Dates

df_cleaned['Signup_Date'] = pd.to_datetime(df_cleaned['Signup_Date'], errors='coerce')

# Why? Different formats and "Not Available" will throw errors. This replaces bad dates with NaT (missing).

In [16]:
# 💰 Step 9: Clean Income Column

# Replace commas and convert to numeric
df_cleaned['Income'] = df_cleaned['Income'].replace('[\$,]', '', regex=True)
df_cleaned['Income'] = pd.to_numeric(df_cleaned['Income'], errors='coerce')

# Fill invalid entries like "fifty thousand" (now NaN) with median
income_median = df_cleaned['Income'].median()
df_cleaned['Income'] = df_cleaned['Income'].fillna(income_median)

#  Why? Numeric columns often contain string symbols, words, or missing data that break models.

In [19]:
# Final Cleaned Data Summary

print(df_cleaned.info())
print(df_cleaned.head())


<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 0 to 5
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Customer_ID  4 non-null      object        
 1   Name         4 non-null      object        
 2   Age          4 non-null      float64       
 3   Email        4 non-null      object        
 4   Signup_Date  3 non-null      datetime64[ns]
 5   Income       4 non-null      float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 224.0+ bytes
None
  Customer_ID   Name   Age              Email Signup_Date   Income
0        C001   John  25.0   john@example.com  2023-01-01  40000.0
1        C002    Ana  30.0      ana@gmail.com         NaT  50000.0
3        C004  Sarah  45.0  sarah@company.com  2023-01-04  40000.0
5        C005   John  30.0   john@example.com  2023-01-01  40000.0
